# Imports

In [11]:
import pandas as pd
import datetime as dt
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.decomposition import PCA
#import xgboost as xg
#from entsoe import EntsoePandasClient
#from statsmodels.graphics import tsaplots
import sys

# Import data

## Entsoe

In [88]:
day_ahead_prices = pd.read_csv(sys.path[0]+"/data/main/DK_2/Day_Ahead_Prices.csv")
day_ahead_prices.tail()

,Unnamed: 0,0
46315,2023-04-14 20:00:00+02:00,152.21
46316,2023-04-14 21:00:00+02:00,151.00
46317,2023-04-14 22:00:00+02:00,134.33
46318,2023-04-14 23:00:00+02:00,82.61
46319,2023-04-15 00:00:00+02:00,123.90


In [89]:
load_and_forecast = pd.read_csv(sys.path[0]+"/data/main/DK_2/Load_And_Forecast.csv")
load_and_forecast.tail()

,Unnamed: 0,Forecasted Load,Actual Load
46314,2023-04-14 19:00:00+02:00,1680.0,1665.0
46315,2023-04-14 20:00:00+02:00,1578.0,1578.0
46316,2023-04-14 21:00:00+02:00,1496.0,1488.0
46317,2023-04-14 22:00:00+02:00,1431.0,1402.0
46318,2023-04-14 23:00:00+02:00,1366.0,1332.0


In [90]:
wind_solar_forecast = pd.read_csv(sys.path[0]+"/data/main/DK_2/Wind_Solar_Forecast.csv")
wind_solar_forecast.tail()

,Unnamed: 0,Solar,Wind Offshore,Wind Onshore
46218,2023-04-14 19:00:00+02:00,12.0,468.0,86.0
46219,2023-04-14 20:00:00+02:00,1.0,497.0,96.0
46220,2023-04-14 21:00:00+02:00,0.0,543.0,108.0
46221,2023-04-14 22:00:00+02:00,0.0,544.0,112.0
46222,2023-04-14 23:00:00+02:00,0.0,546.0,105.0


## Imports

#### Apply Remove UTC functions

In [109]:
imports = pd.read_csv(sys.path[0]+"/data/imports/DK_2/Imports.csv")

In [92]:
imports.head()

,Unnamed: 0,DE_AT_LU,DE_LU,DK_1,SE_4
0,2018-01-01 00:00:00+01:00,600.0,NaN,585.0,0.0
1,2018-01-01 01:00:00+01:00,600.0,NaN,590.0,0.0
2,2018-01-01 02:00:00+01:00,600.0,NaN,476.0,0.0
3,2018-01-01 03:00:00+01:00,600.0,NaN,538.0,0.0
4,2018-01-01 04:00:00+01:00,600.0,NaN,520.0,0.0


In [41]:
imports['Unnamed: 0'] = pd.to_datetime(imports['Unnamed: 0']).apply(lambda row: row.replace(tzinfo=None))

In [42]:
imports.set_index("Unnamed: 0").r

,Unnamed: 0,DE_AT_LU,DE_LU,DK_1,SE_4
0,2018-01-01 00:00:00,600.0,NaN,585.0,0.0
1,2018-01-01 01:00:00,600.0,NaN,590.0,0.0
2,2018-01-01 02:00:00,600.0,NaN,476.0,0.0
3,2018-01-01 03:00:00,600.0,NaN,538.0,0.0
4,2018-01-01 04:00:00,600.0,NaN,520.0,0.0


In [102]:
imports['Timestamp'] = pd.to_datetime(imports['Unnamed: 0'], format = '%Y %m %d %H:%M:%S',utc = True)
imports.head()

,Unnamed: 0,DE_AT_LU,DE_LU,DK_1,SE_4,Timestamp
0,2018-01-01 00:00:00+01:00,600.0,NaN,585.0,0.0,2017-12-31 23:00:00+00:00
1,2018-01-01 01:00:00+01:00,600.0,NaN,590.0,0.0,2018-01-01 00:00:00+00:00
2,2018-01-01 02:00:00+01:00,600.0,NaN,476.0,0.0,2018-01-01 01:00:00+00:00
3,2018-01-01 03:00:00+01:00,600.0,NaN,538.0,0.0,2018-01-01 02:00:00+00:00
4,2018-01-01 04:00:00+01:00,600.0,NaN,520.0,0.0,2018-01-01 03:00:00+00:00


In [103]:
imports['Timestamp'] = (imports['Timestamp'] + dt.timedelta(hours = 2)).dt.tz_localize(None)
imports.head()

,Unnamed: 0,DE_AT_LU,DE_LU,DK_1,SE_4,Timestamp
0,2018-01-01 00:00:00+01:00,600.0,NaN,585.0,0.0,2018-01-01 01:00:00
1,2018-01-01 01:00:00+01:00,600.0,NaN,590.0,0.0,2018-01-01 02:00:00
2,2018-01-01 02:00:00+01:00,600.0,NaN,476.0,0.0,2018-01-01 03:00:00
3,2018-01-01 03:00:00+01:00,600.0,NaN,538.0,0.0,2018-01-01 04:00:00
4,2018-01-01 04:00:00+01:00,600.0,NaN,520.0,0.0,2018-01-01 05:00:00


In [108]:
imports[imports['Timestamp'].date()=='20230313']

AttributeError: 'Series' object has no attribute 'date'

In [110]:
imports = remove_utc('Unnamed: 0', 2, imports)

In [112]:
imports.tail(50)

,DE_AT_LU,DE_LU,DK_1,SE_4
Timestamp,,,,
2023-04-14 11:30:00,NaN,0.0,NaN,NaN
2023-04-14 11:45:00,NaN,0.0,NaN,NaN
2023-04-14 12:00:00,NaN,0.0,212.0,1237.0
2023-04-14 12:15:00,NaN,0.0,NaN,NaN
2023-04-14 12:30:00,NaN,0.0,NaN,NaN
2023-04-14 12:45:00,NaN,0.0,NaN,NaN
2023-04-14 13:00:00,NaN,0.0,250.0,1252.0
2023-04-14 13:15:00,NaN,0.0,NaN,NaN
2023-04-14 13:30:00,NaN,0.0,NaN,NaN


In [119]:
imports[(imports.index.year == 2023) & (imports.index.month==4) & (imports.index.day==13)].head(50)

,DE_AT_LU,DE_LU,DK_1,SE_4
Timestamp,,,,
2023-04-13 00:00:00,NaN,0.0,0.0,1081.0
2023-04-13 00:15:00,NaN,0.0,NaN,NaN
2023-04-13 00:30:00,NaN,0.0,NaN,NaN
2023-04-13 00:45:00,NaN,0.0,NaN,NaN
2023-04-13 01:00:00,NaN,0.0,0.0,1167.0
2023-04-13 01:15:00,NaN,0.0,NaN,NaN
2023-04-13 01:30:00,NaN,0.0,NaN,NaN
2023-04-13 01:45:00,NaN,0.0,NaN,NaN
2023-04-13 02:00:00,NaN,0.0,0.0,1169.0


#### Group by Hour because they have it by 15 mins

In [74]:
importshour = imports.groupby(pd.Grouper(freq='H')).sum()
#importshour

#imports_columns = list(importshour.columns)
#imports_columns

## Exports

#### Read CSV

In [87]:
import os

def find_file(file_name, search_path):
    """Search for a file with a specific name in the given search path."""
    for dir_path, _, file_list in os.walk(search_path):
        # Look for the file in the current directory
        if file_name in file_list:
            return os.path.join(dir_path, file_name)

    # If the file was not found
    return None

# Specify the file name and search path
file_name = "Exports.csv"
search_path = sys.path[0]

# Call the function to find the file
result = find_file(file_name, search_path)

# Check if the file was found or not
if result:
    print("File found at:", result)
else:
    print("File not found.")


'/home/gpanagioto/GitHub/AdvBA/EPF_ABA23/data/exports/DK_2/SE_4/Exports.csv'

In [98]:
exDELU = pd.read_csv(find_file('DE_LU_Exports.csv',search_path))
exDK1 = pd.read_csv(find_file('DK_1_Exports.csv', search_path))
exSE4 = pd.read_csv(find_file('SE_4_Exports.csv', search_path))

In [99]:
exDELU.head()

,Unnamed: 0,0
0,2018-10-01 00:00:00+02:00,365.0
1,2018-10-01 01:00:00+02:00,538.0
2,2018-10-01 02:00:00+02:00,550.0
3,2018-10-01 03:00:00+02:00,584.0
4,2018-10-01 04:00:00+02:00,585.0


In [100]:
exDK1.head()

,Unnamed: 0,0
0,2017-12-24 00:00:00+01:00,0.0
1,2017-12-24 01:00:00+01:00,0.0
2,2017-12-24 02:00:00+01:00,0.0
3,2017-12-24 03:00:00+01:00,0.0
4,2017-12-24 04:00:00+01:00,0.0


In [101]:
exSE4.head()

,Unnamed: 0,0
0,2017-12-24 00:00:00+01:00,535.0
1,2017-12-24 01:00:00+01:00,750.0
2,2017-12-24 02:00:00+01:00,831.0
3,2017-12-24 03:00:00+01:00,650.0
4,2017-12-24 04:00:00+01:00,457.0


#### Apply Time Removal Function and makes it a Timestamp

In [180]:
exDELU = remove_utc('Unnamed: 0', 1, exDELU)
exDK1 = remove_utc('Unnamed: 0', 1, exDK1)
exSE4 = remove_utc('Unnamed: 0', 1, exSE4)

#### Make a groupby for each hour so it has the same format

In [181]:
exDELU = exDELU.groupby(pd.Grouper(freq='H')).sum()
exDK1 = exDK1.groupby(pd.Grouper(freq='H')).sum()
exSE4 = exSE4.groupby(pd.Grouper(freq='H')).sum()

#### Change columns to export specifics

In [182]:
exDELU.rename(columns = {'0':'Export De-Lu'}, inplace = True)
exDK1.rename(columns = {'0':'Export DK1'}, inplace = True)
exSE4.rename(columns = {'0':'Export SE4'}, inplace = True)

## Investing

In [184]:
# Natural Gas - TTF prices
ttf_prices = pd.read_csv('Dutch TTF Natural Gas Futures Historical Data.csv', decimal = '.')  # in €
ttf_prices['Date'] = pd.to_datetime(ttf_prices['Date'], format = '%m/%d/%Y') # convert date from string to datetime object

# drop unnecessary columns
cols_drop = [col for col in ttf_prices.columns if (col != 'Date' and col != 'Price')]
ttf_prices.drop(cols_drop, axis = 1, inplace = True)

ttf_prices.sort_values(by = ['Date'], ascending = True, inplace = True)
ttf_prices.reset_index(inplace = True, drop = True)

In [213]:
#ttf_prices

In [186]:
# CO2 emissions prices
co2_prices = pd.read_csv('Carbon Emissions Futures Historical Data.csv', decimal = '.')  # in €
co2_prices['Date'] = pd.to_datetime(co2_prices['Date'], format = '%m/%d/%Y') # convert date from string to datetime object

# drop unnecessary columns
cols_drop = [col for col in co2_prices.columns if (col != 'Date' and col != 'Price')]
co2_prices.drop(cols_drop, axis = 1, inplace = True)

co2_prices.sort_values(by = ['Date'], ascending = True, inplace = True)
co2_prices.reset_index(inplace = True, drop = True)

In [214]:
#co2_prices.head()

#co2_prices.tail()

# Pre-processing
## Adjust timeseries

In [34]:
def remove_utc(col_name, # str: name of the column that contains the object to convert to timestamp
               tz_offset, # int: timezone offset. E.g., CET = +1
               df # dataframe: contains all the info
              ):
    df['Timestamp'] = pd.to_datetime(df[col_name], format = '%Y %m %d %H:%M:%S',utc = True)
    df['Timestamp'] = (df['Timestamp'] + dt.timedelta(hours = tz_offset)).dt.tz_localize(None)
    df.drop([col_name], axis = 1, inplace = True) # drop the column
    df.set_index('Timestamp', inplace = True) # set column 'Timestamp' as index
    return df

In [57]:
day_ahead_prices2 = remove_utc('Unnamed: 0', 1, day_ahead_prices)

In [66]:
day_ahead_prices2.sort_index().head()

,0
Timestamp,
2018-01-01 00:00:00,26.33
2018-01-01 01:00:00,26.43
2018-01-01 02:00:00,26.10
2018-01-01 03:00:00,24.70
2018-01-01 04:00:00,24.74


In [63]:
day_ahead_prices.head()

,0
Timestamp,
2018-01-01 00:00:00,26.33
2018-01-01 01:00:00,26.43
2018-01-01 02:00:00,26.10
2018-01-01 03:00:00,24.70
2018-01-01 04:00:00,24.74


In [217]:
load_and_forecast  = remove_utc('Unnamed: 0', 1, load_and_forecast)

In [ ]:
load_and_forecast.tail()

In [193]:
wind_solar_forecast = remove_utc('Unnamed: 0', 1, wind_solar_forecast)

In [218]:
#wind_solar_forecast.tail()
#wind_solar_forecast.info()

## Join datasets

In [195]:
# create empty dataframe where to "pour" all the data
start = pd.to_datetime('2018-01-01 00:00:00')
end = pd.to_datetime('2023-03-31 23:59:00')
df = pd.DataFrame()
df['Timestamp'] = pd.date_range(start, end, freq = 'H')
df['Date'] = df['Timestamp'].dt.date
df.set_index('Timestamp', inplace = True)# set column 'Timestamp' as index

In [219]:
#df

In [197]:
# mapping daily prices: TTF and CO2
df['TTF'] = df['Date'].map(ttf_prices.set_index('Date')['Price'])
df['CO2'] = df['Date'].map(co2_prices.set_index('Date')['Price'])

In [220]:
#df

In [202]:
df_merged = pd.merge(df, day_ahead_prices, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, load_and_forecast, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, wind_solar_forecast, on = 'Timestamp', how = 'outer')

df_merged = pd.merge(df_merged, importshour, on = 'Timestamp', how = 'outer')

df_merged = pd.merge(df_merged, exDELU, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, exDK1, on = 'Timestamp', how = 'outer')
df_merged = pd.merge(df_merged, exSE4, on = 'Timestamp', how = 'outer')


df_merged.reset_index(inplace = True)

,Timestamp,Date,TTF,CO2,Day-ahead prices,Forecasted Load,Actual Load,Solar,Wind Offshore,Wind Onshore,DE_AT_LU,DE_LU,DK_1,SE_4,Export De-Lu,Export DK1,Export SE4
45934,2023-03-29 22:00:00,2023-03-29,42.805,90.02,94.80,1403.0,1390.0,0.0,54.0,29.0,0.0,0.0,0.0,1261.0,2406.0,176.0,0.0
45935,2023-03-29 23:00:00,2023-03-29,42.805,90.02,89.71,1324.0,1352.0,0.0,146.0,45.0,0.0,0.0,0.0,1230.0,2504.0,202.0,0.0
45936,2023-03-30 00:00:00,2023-03-30,43.609,90.80,82.25,1301.0,1343.0,0.0,159.0,56.0,0.0,0.0,0.0,1265.0,2197.0,346.0,0.0
45937,2023-03-30 01:00:00,2023-03-30,43.609,90.80,80.75,1309.0,1316.0,0.0,178.0,67.0,0.0,0.0,0.0,1192.0,2303.0,334.0,0.0
45938,2023-03-30 02:00:00,2023-03-30,43.609,90.80,78.77,1334.0,1301.0,0.0,244.0,98.0,0.0,0.0,0.0,1188.0,1976.0,500.0,0.0
45939,2023-03-30 03:00:00,2023-03-30,43.609,90.80,77.91,1335.0,1283.0,0.0,324.0,145.0,0.0,0.0,0.0,825.0,960.0,465.0,0.0
45940,2023-03-30 04:00:00,2023-03-30,43.609,90.80,76.80,1370.0,1341.0,0.0,367.0,200.0,0.0,0.0,16.0,543.0,1354.0,119.0,0.0
45941,2023-03-30 05:00:00,2023-03-30,43.609,90.80,89.93,1518.0,1524.0,0.0,442.0,264.0,0.0,1.0,476.0,23.0,854.0,0.0,53.0
45942,2023-03-30 06:00:00,2023-03-30,43.609,90.80,119.21,1699.0,1702.0,2.0,533.0,296.0,0.0,0.0,587.0,3.0,1633.0,0.0,37.0
45943,2023-03-30 07:00:00,2023-03-30,43.609,90.80,129.30,1784.0,1816.0,10.0,637.0,296.0,0.0,0.0,588.0,0.0,1257.0,0.0,337.0


In [ ]:
#check the state of D f
df_merged.tail(50)

In [203]:
# add wind generation columns
df_merged['Wind Total'] = df_merged['Wind Offshore'] + df_merged['Wind Onshore']

In [204]:
#df_merged
df_merged.columns

Index(['Timestamp', 'Date', 'TTF', 'CO2', 'Day-ahead prices',
       'Forecasted Load', 'Actual Load', 'Solar', 'Wind Offshore',
       'Wind Onshore', 'DE_AT_LU', 'DE_LU', 'DK_1', 'SE_4', 'Export De-Lu',
       'Export DK1', 'Export SE4', 'Wind Total'],
      dtype='object')

So the DE_LU electricity exchange started from 10-01-2018 so before that we fill it with 0s, and before that it was DE_AT_LU which stops at 30-01-2018 and so we ignore it.

In [221]:
df_merged['Export De-Lu'].isnull().sum()

mask = (df_merged['Timestamp'] >= '2018-01-01 00:00:00') & (df_merged['Timestamp'] < '2018-09-30 23:00:00')
df_merged.loc[mask,'Export De-Lu'] = df_merged.loc[mask,'Export De-Lu'].fillna(0)

25

## Saving the resulting dataset

In [212]:
df_merged.to_csv('DK_2_imp_exp.csv', index = False)

In [222]:
#df_merged